# Import

In [1]:
import torch
from torch import FloatTensor, LongTensor
import math

In [2]:
from load_script_deep_framework import load_dataset

In [9]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [11]:
train,target_train=load_dataset()

In [12]:
test,target_test=load_dataset()